# Advent of Code 2024 - J12

In [1]:
def read_input(kind):
    assert kind in ('input', 'example', 'example1', 'example2'), '"kind" must be "input" or "example" or "example1" or "example2"'
    garden = []
    with open(kind) as f:
        for line in f:
            garden.append(line.strip())
    l = len(garden[0])
    for line in garden:
        assert len(line) == l, "input should be a rectangle"
    return garden

In [2]:
def print_garden(garden):
    colors = [
        '\033[30m', '\033[31m', '\033[32m', '\033[33m',
        '\033[34m', '\033[35m', '\033[36m', '\033[37m',
        '\033[90m', '\033[91m', '\033[92m', '\033[93m',
        '\033[94m', '\033[95m', '\033[96m', '\033[97m']
    letters = {}
    current = 0
    for line in garden:
        l = ''
        for c in line:
            if c not in letters:
                letters[c] = colors[current]
                current = (current + 1)%len(colors)
            l += (letters[c]+c)
        print(l)

In [3]:
print_garden(read_input("example1"))

AAAA
BBCD
BBCC
EEEC


In [4]:
print_garden(read_input("example2"))

OOOOO
OXOXO
OOOOO
OXOXO
OOOOO


In [5]:
print_garden(read_input("example"))

RRRRIICCFF
RRRRIICCCF
VVRRRCCFFF
VVRCCCJFFF
VVVVCJJCFE
VVIVCCJJEE
VVIIICJJEE
MIIIIIJJEE
MIIISIJEEE
MMMISSJEEE


In [6]:
print_garden(read_input("input"))

IIIIIIIIIIIIIIIUUUUUUUUUUUUUUUUUUUSSAAAAAAAAAAAOFFFFFFFFFFFFCCCNNNKKKKKKKKKKKGGGGGGGHHHHHHHHCCRXXXXXXXXIIIILKLLLLLLLLLLLLLLLLFTTTTTTTTTTWWWW
IIAIIIYIIIIIIIIIUUUUUUUUUUUUUUUUUUUSUAAAAAAAAAAFFFFFFFFFEFFCCCCNNNKKKKKKKKGGGGGGNGGGHHHHHHHCCCXXXXXXXXXIIIILLLLLLLLLLLLLLLLLLFTTTTTTTTTTWWXW
IIAIIIYYYYYYYYYYYUUUUUUUUUUUUUUUUUUUUAAAAAAAAAAFFEEEFFEEEENNNNNNNNKKKKKKGGGGGGGGGGGGGGGHHHHCCCCXCXXXXXIIIILLLLLLLLLLLLLLLLLLLFTTTTTTTTTWWXXW
IAAAIIYYYYYYYYYYLUUUUUUUUUHHUUUUUUUUAAAAAAAAAAAFEEEEEFEEEENNNNNNNNNKKKKKKGGGGGGGGGGGGGGHHHCCCCCCXXXXXXIMIMLLMLLLLLLLLLLLLLLLFFFFFTTTTTTXXXXX
AAAIIIYYYYYYYYYYUUUUUUUUUUHHHHHHUUUUUUUAAAAAAAAAEEEEEEEEEEEWNNNNZZZZZKKKGGGGGGGGGGGGGGGGGCCCCCCCCXXXXXIMMMLLMLLLLLLLLLLLLLLLLFFTTTTTTTTXXXXX
AAAIIYYYYYYYYYYYYUUUUUUUUHHHHHHHHHHUUUAAAAAAAAAAEEEEEEEEEEENNNNZZZZZZZZKKKGGGGGGGGGGGGGGWCCCCCCCXXXXXXMMMMLMMMMLLLLLLLLLLLLLFFFTTTTTTTTXXXXQ
AAAIIIYRRYYYYYYYYUUUUHHHHHHHHHHHHHHAAUAAAAAAAAAEEEOEEEEEEEEENNNNNZZZKKKKKKKGGGGGGGGGGGGGWCCCCCCCRXXXJJMMMMMMMMMLLLLLLLLLLLLFFRTTTTTXXXXXXXXX
AAAAIIARRRYYY

## First part

Multiply area by perimeter for the price of each _garden plot_ and add all prices.

example1: 4×10 + 4×8 + 4×10 + 1×4 + 3×8 = **140**\
example2: 21×36 + (1×4)×4 = **772**\
example: **1930**

In [7]:
def tmap_to_str(tmap):
    output = []
    for line in tmap:
        l = ''
        for c in line:
            l += str(c)
        output.append(l)
    return output

def print_garden_and_explored(garden, explored):
    str_explored = tmap_to_str(explored)
    assert len(garden) == len(str_explored), "'garden' and 'explored' should have the same number of lines"
    assert len(garden[0]) == len(str_explored[0]), "'garden' and 'explored' should have the same number of columns"
    colors = [
        '\033[30m', '\033[31m', '\033[32m', '\033[33m',
        '\033[34m', '\033[35m', '\033[36m', '\033[37m',
        '\033[90m', '\033[91m', '\033[92m', '\033[93m',
        '\033[94m', '\033[95m', '\033[96m', '\033[97m']
    letters = {}
    current = 0
    for i in range(len(garden)):
        l = ''
        for c in garden[i]:
            if c not in letters:
                letters[c] = colors[current]
                current = (current + 1)%len(colors)
            l += (letters[c]+c)
        l += '  '
        for c in str_explored[i]:
            if c not in letters:
                letters[c] = colors[current]
                current = (current + 1)%len(colors)
            l += (letters[c]+c)
        print(l)

In [8]:
from collections import deque

def first_part(kind, p=False):
    garden = read_input(kind)
    nb_lines = len(garden)
    nb_cols = len(garden[0])
    explored = [[0 for _ in range(nb_cols)] for _ in range(nb_lines)]
    if p:
        print_garden_and_explored(garden, explored)
        print('\n')
    directions = [(1, 0), (-1, 0), (0, 1), (0, -1)]
    price = 0
    for y, line in enumerate(garden):
        for x, car in enumerate(line):
            if explored[y][x] == 0: # Plant not explored yet
                current_plot = {
                    'letter': car,
                    'perimeter': 0,
                    'area': 0
                }
                d = deque()
                d.append((y, x))
                while d:
                    # print(d)
                    cy, cx = d.popleft()
                    # Check if already explored or not:
                    if explored[cy][cx] == 1:
                        # If yes, we ignore it and go to next plant
                        continue
                    # We add 1 to area and mark it as explored
                    current_plot['area'] += 1
                    explored[cy][cx] = 1
                    # Then we look for all its neighbours that have the same letter
                    neighbours = [(cy + d[0], cx + d[1]) for d in directions]
                    for ny, nx in neighbours:
                        # First check if ny is within garden bounds:
                        if not 0 <= ny < nb_lines:
                            # Then we have a border
                            current_plot['perimeter'] += 1
                            # And we go to next neighbour
                            continue
                        # Then same with nx:
                        if not 0 <= nx < nb_cols:
                            # Then we have a border
                            current_plot['perimeter'] += 1
                            # And we go to next neighbour
                            continue
                        # If (ny, nx) is within garden bounds:
                        # If no (so (ny, nx) is within garden bounds, and never explored), then check if it's the same letter:
                        if garden[ny][nx] == current_plot['letter']:
                            # If yes, add the neighbour to d:
                            d.append((ny, nx))
                        else:
                            # If no, then we have a border
                            current_plot['perimeter'] += 1
                    if p:
                        print(current_plot)
                        print(d)
                        print_garden_and_explored(garden, explored)
                        print('\n')
                # When d is empty, we have covered all the garden plot, so we can add its price to the global price
                price += current_plot['area'] * current_plot['perimeter']
    return price

In [9]:
first_part("example1")

140

In [10]:
first_part("example2")

772

In [11]:
first_part("example")

1930

In [12]:
first_part("input")

1375574

## Second part